In [38]:
import spacy
from spacy.tokens import Span
from spacy.matcher import PhraseMatcher

nlp = spacy.load("en_core_web_sm")

animals = ["Golden Retriever", "cat", "turtle", "Rattus norvegicus"]
matcher = PhraseMatcher(nlp.vocab)

In [ ]:
def animal_component(doc):
    animals = ["Golden Retriever", "cat", "turtle", "Rattus norvegicus"]
    matcher = PhraseMatcher(nlp.vocab)
    
    matches = matcher(doc)
    span = [Span(doc, start, end, label="ANIMAL") for match_id, start, end in matches]
    doc.ents = span
    return doc

In [ ]:
global nlp
nlp.add_pipe(animal_component, after=True)

In [43]:
def get_entities(doc):
    return [(ent.text, ent.label_) for ent in doc.ents if ent.label_ == "ORG"]

def get_countries(doc):
    return [(ent.text, ent.label_) for ent in doc.ents if ent.label_ == 'GPE']

def get_persons(doc):
    return [(ent.text, ent.label_) for ent in doc.ents if ent.label_ == 'PERSON']


In [44]:
TEXT = """
20 Newsgroups: 20,000 documents from over 20 different newsgroups. 
The content covers a variety of topics with some closely related for 
reference. There are three versions, one in its original form, 
one with dates removed, and one with duplicates removed. I like David Bowie.
It’s official: Apple is the first U.S. public company to reach a $1 trillion market value
"""

doc = nlp(TEXT)

print('Entities:   ', get_entities(doc))
print('Countries   :',  get_countries(doc))
print('PERSON   :',  get_persons(doc))

Entities:    [('Apple', 'ORG')]
Countries   : [('U.S.', 'GPE')]
PERSON   : [('David Bowie', 'PERSON')]
